In [11]:
import cv2
import os
import numpy as np
import tensorflow as tf

# Define the path to your video files and the output directory
video_folder = 'C:/Users/ALIN/OneDrive/Desktop/HSL/videoSamples/2. quiet'
output_directory = 'C:/Users/ALIN/OneDrive/Desktop/HSL/MP_DATA/quiet'

In [18]:
# List of video file names (e.g., video1.mov, video2.mov, etc.)
video_files = ['MVI_5180.MOV','MVI_5181.MOV','MVI_5182.MOV','MVI_5260.MOV','MVI_5261.MOV','MVI_5262.MOV','MVI_5338.MOV',
               'MVI_5339.MOV','MVI_5340.MOV','MVI_9292.MOV','MVI_9293.MOV','MVI_9294.MOV','MVI_9371.MOV','MVI_9372.MOV',
               'MVI_9373.MOV','MVI_9451.MOV','MVI_9452.MOV','MVI_9452.MOV','MVI_9453.MOV','MVI_9537.MOV','MVI_9538.MOV','MVI_9539.MOV']

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Function to extract frames from a video and save them as .np files
def extract_frames(video_path, output_folder, video_name, num_frames=30):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = frame_count // num_frames

    # Create a subfolder for each video
    video_output_folder = os.path.join(output_folder, video_name)
    if not os.path.exists(video_output_folder):
        os.makedirs(video_output_folder)

    frame_number = 0
    frame_index = 0

    while frame_index < num_frames:
        success, frame = cap.read()
        if not success:
            break
        if frame_number % frame_interval == 0:
            frame_filename = os.path.join(video_output_folder, f'{frame_index}.npy')
            # Convert the frame to a TensorFlow tensor
            frame_tensor = tf.convert_to_tensor(frame, dtype=tf.uint8)
            # Save the TensorFlow tensor as .npy
            np.save(frame_filename, frame_tensor)
            frame_index += 1
        frame_number += 1

    cap.release()

# Process each video file
for video_file in video_files:
    video_path = os.path.join(video_folder, video_file)
    video_name = os.path.splitext(video_file)[0]
    extract_frames(video_path, output_directory, video_name)

print("Frames extraction completed.")

Frames extraction completed.


In [17]:
import cv2
import os
import numpy as np
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Define the input folder containing video filesdddd
input_folder = 'C:/Users/ALIN/OneDrive/Desktop/HSL/videoSamples/2. quiet'

# Define the output folder where frames will be saved
output_directory = 'C:/Users/ALIN/OneDrive/Desktop/HSL/MP_DATA/quiet'

# Create the output directory if it doesn't exist
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

# Set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    # List video files in the input folder
    video_files = [f for f in os.listdir(input_folder) if f.endswith('.mov')]

    for video_file in video_files:
        video_file_path = os.path.join(input_folder, video_file)

        # Initialize video capture from the video file
        cap = cv2.VideoCapture(video_file_path)

        frame_num = 0  # Initialize frame number

        while True:
            ret, frame = cap.read()

            if not ret:
                break

            # Make detections
            image, results = mediapipe_detection(frame, holistic)
            print(results)

            # Draw Landmarks
            draw_styled_landmarks(image, results)

            if frame_num == 0:
                cv2.putText(image, 'STARTING COLLECTION', (120, 200),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4, cv2.LINE_AA)
                cv2.putText(image, 'Collecting frames for Video', (15, 12),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                cv2.waitKey(2000)
            else:
                cv2.putText(image, 'Collecting frames for Video', (15, 12),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)

            keypoints = extract_keypoints(results)
            npy_path = os.path.join(output_directory, f'{video_file}_{frame_num}.npy')
            np.save(npy_path, keypoints)

            # Show to screen
            cv2.imshow('OpenCV Feed', image)

            frame_num += 1

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()

print("Frames extraction completed.")


Frames extraction completed.
